In [ ]:
# there's no reason why this is called demeter
# only that i feel as cranky as the goddess of seasons rn

In [14]:
import h5py

path = r"C:\Users\jerom\Documents\GitHub\WGANSing\ss_synthesis\old_voice\nus_ADIZ_sing_01.hdf5"

f = h5py.File(path, "r")

for key in f.keys():
    print(key)

    group = f[key]
    print(group)

    print(group[0])

feats
<HDF5 dataset "feats": shape (10963, 66), type "<f4">
[-7.66702423e+01 -7.65191422e+01 -7.67861328e+01 -7.80998840e+01
 -8.03749084e+01 -8.26100616e+01 -8.50882721e+01 -8.76643677e+01
 -9.11775208e+01 -9.35921478e+01 -9.33911743e+01 -9.14124527e+01
 -8.93775330e+01 -9.01215210e+01 -9.22904587e+01 -9.32163467e+01
 -9.29562759e+01 -9.33782120e+01 -9.58699341e+01 -9.79580688e+01
 -9.77253265e+01 -9.44750443e+01 -9.09774399e+01 -8.98304977e+01
 -9.32809830e+01 -9.84550552e+01 -1.01191025e+02 -9.75223007e+01
 -9.04919662e+01 -8.67748032e+01 -9.20113220e+01 -9.86948700e+01
 -9.71751862e+01 -9.48190689e+01 -9.42237015e+01 -9.57077713e+01
 -9.61584473e+01 -9.40252686e+01 -9.84906387e+01 -9.66487656e+01
 -9.86617737e+01 -9.60188751e+01 -1.01515701e+02 -9.48090591e+01
 -8.97704544e+01 -9.33054428e+01 -1.11652504e+02 -1.04285912e+02
 -9.95474930e+01 -1.02036560e+02 -1.00222481e+02 -1.03889702e+02
 -1.00331459e+02 -9.44626923e+01 -9.73388443e+01 -1.10695145e+02
 -9.95999146e+01 -1.01423210e+

In [3]:
## What the fuck is in h5py? discuss.

import h5py

path = r"stats\stats.hdf5"

f = h5py.File(path, "r")

for key in f.keys():
    print(key)

    group = f[key]
    print(group)

    print(group[0:10])

feats_maximus
<HDF5 dataset "feats_maximus": shape (66,), type "<f4">
[ 7.106183   8.1570215  7.967522  11.031192  13.544613  10.586577
  8.571584  11.770323  11.43437   12.688358 ]
feats_minimus
<HDF5 dataset "feats_minimus": shape (66,), type "<f4">
[-120. -120. -120. -120. -120. -120. -120. -120. -120. -120.]


In [24]:
import config

In [26]:
voice_dir = "ss_synthesis/voice/"

In [57]:
# A GENERATOR FUNCTION, THE GENERATED OUTPUTS CHANGE AS THE CODE PROGRESSES
# MORE SAMPLES ARE ADDED FOR EACH "BATCH" COMPLETED

# Gets a fixed number of files to process based on the batch size
# Then aggregates/concats TO INTERNAL LIST/state over entire dataset
def data_gen(mode = 'Train', sec_mode = 0):
    voc_list = [x for x in os.listdir(voice_dir) if x.endswith('.hdf5') and x.startswith('nus')]
    # val_list = ['nus_MCUR_sing_04.hdf5', 'nus_ADIZ_read_01.hdf5', 'nus_JLEE_sing_05.hdf5','nus_JTAN_read_07.hdf5' ]

    # import pdb;pdb.set_trace()

    stat_file = h5py.File(config.stat_dir+'stats.hdf5', mode='r')

    max_feat = np.array(stat_file["feats_maximus"])
    min_feat = np.array(stat_file["feats_minimus"])

    stat_file.close()


    max_files_to_process = int(config.batch_size/config.samples_per_file)
    # max_files_to_process = 1

    if mode == "Train":
        num_batches = config.batches_per_epoch_train
        if sec_mode == 0:
            file_list = voc_list

    else: 
        # num_batches = config.batches_per_epoch_val
        # file_list = val_list
        return

    for k in range(num_batches):
        if sec_mode == 1:
            if np.random.rand(1)<config.aug_prob:
                file_list = voc_list
            else:
                file_list = voc_list
        

        feats_targs = []
        targets_f0_1 = []
        targets_singers = []
        pho_targs = []

        # start_time = time.time()
        if k == num_batches-1 and mode =="Train":
            file_list = voc_list

        for i in range(max_files_to_process):
            #randomly choose some file to process
            voc_index = np.random.randint(0,len(file_list))
            voc_to_open = file_list[voc_index]
            voc_file = h5py.File(config.voice_dir+voc_to_open, "r")
            feats = np.array(voc_file['feats'])

            f0 = feats[:,-2]

            med = np.median(f0[f0 > 0])

            f0[f0==0] = med

            f0_nor = (f0 - min_feat[-2])/(max_feat[-2]-min_feat[-2])
            

            feats = (feats-min_feat)/(max_feat-min_feat)

            feats[:,-2] = f0_nor


            if voc_to_open.startswith('nus'):
                if not  "phonemes" in voc_file:
                    # print(voc_file)
                    Flag = False
                else: 
                    Flag = True
                    pho_target = np.array(voc_file["phonemes"])
                    singer_name = voc_to_open.split('_')[1]
                    singer_index = config.singers.index(singer_name)
                    # print("singer", singer_name, singer_index)
            else:
                Flag = False

            # there are 6 samples per file
            for j in range(config.samples_per_file):
                    # randomly get a WINDOW of 128 frames for each file.
                    # each 128 frames is a sample
                    voc_idx = np.random.randint(0,len(feats)-config.max_phr_len)
                    # print("start", voc_idx, "end", voc_idx+config.max_phr_len)
                    targets_f0_1.append(f0_nor[voc_idx:voc_idx+config.max_phr_len])
                    if Flag:
                        pho_targs.append(pho_target[voc_idx:voc_idx+config.max_phr_len])
                        targets_singers.append(singer_index)

                    feats_targs.append(feats[voc_idx:voc_idx+config.max_phr_len])

        targets_f0_1 = np.expand_dims(np.array(targets_f0_1), -1)

        feats_targs = np.array(feats_targs)

        # assert feats_targs.max()<=1.0 and feats_targs.min()>=0.0

        yield feats_targs, targets_f0_1, np.array(pho_targs), np.array(targets_singers)

In [85]:
import numpy as np
import os
import h5py

sample_threshold = 20
# A GENERATOR FUNCTION, THE GENERATED OUTPUTS CHANGE AS THE CODE PROGRESSES
# MORE SAMPLES ARE ADDED FOR EACH "BATCH" COMPLETED

# Gets a fixed number of files to process based on the batch size
# Then aggregates/concats TO INTERNAL LIST/state over entire dataset
def data_gen_new():

    ## ADJUST WHATEVER IS IN VOC_LIST TO CHANGE NUMBER OF SAMPLES
    voc_list = [x for x in os.listdir(voice_dir) if x.endswith('.hdf5') and x.startswith('nus')]
    # print(voc_list)

    stat_file = h5py.File(config.stat_dir+'stats.hdf5', mode='r')
    max_feat = np.array(stat_file["feats_maximus"])
    min_feat = np.array(stat_file["feats_minimus"])

    stat_file.close()

    max_files_to_process = int(config.batch_size/config.samples_per_file)
    # max_files_to_process = 1

    num_batches = config.batches_per_epoch_train
    file_list = voc_list

    # The outside number of batches is PAUSED for each iteration (k)
    for k in range(num_batches):
        feats_targs = []
        targets_f0_1 = []
        targets_singers = []
        pho_targs = []

        # maximum of about 5 files? to process
        while len(feats_targs) < config.batch_size:
            #randomly choose some file to process
            voc_index = np.random.randint(0,len(file_list))
            voc_to_open = file_list[voc_index]
            voc_file = h5py.File(voice_dir + voc_to_open, "r")
            feats = np.array(voc_file['feats'])

            f0 = feats[:,-2]
            med = np.median(f0[f0 > 0])
            f0[f0==0] = med
            f0_nor = (f0 - min_feat[-2])/(max_feat[-2]-min_feat[-2])

            feats = (feats-min_feat)/(max_feat-min_feat)

            feats[:,-2] = f0_nor


            if voc_to_open.startswith('nus'):
                if not  "phonemes" in voc_file:
                    print(voc_file)
                    Flag = False
                else: 
                    Flag = True
                    pho_target = np.array(voc_file["phonemes"])
                    singer_name = voc_to_open.split('_')[1]
                    singer_index = config.singers.index(singer_name)
            else:
                Flag = False

            # there are 6 samples per file
            for j in range(config.samples_per_file):
                # randomly get a WINDOW of 128 frames for each file.
                # each 128 frames is a sample
                broken = False
                voc_idx = np.random.randint(0,len(feats)-config.max_phr_len)
                
                feat = feats[voc_idx:voc_idx+config.max_phr_len]
                resample_count = 0
                while feat.max()>1.0 or feat.min()<0.0:
                    resample_count += 1
                    voc_idx = np.random.randint(0,len(feats)-config.max_phr_len)
                    feat = feats[voc_idx:voc_idx+config.max_phr_len]
                    if resample_count > 20:
                        broken = True
                        print(singer_name)
                        break
                
                if broken:
                    break
                
                targets_f0_1.append(f0_nor[voc_idx:voc_idx+config.max_phr_len])
                if Flag:
                    pho_targs.append(pho_target[voc_idx:voc_idx+config.max_phr_len])
                    targets_singers.append(singer_index)

                assert feat.max()<=1.0 and feat.min()>=0.0, "chicken nigget" 
                feats_targs.append(feat)

                if len(feats_targs) == config.batch_size:
                    break

            print(len(feats_targs))

        # print(len(feats_targs))
                        

        targets_f0_1 = np.expand_dims(np.array(targets_f0_1), -1)

        feats_targs = np.array(feats_targs)

        assert feats_targs.max()<=1.0 and feats_targs.min()>=0.0, "offending singers: " + str(targets_singers) 

        yield feats_targs, targets_f0_1, np.array(pho_targs), np.array(targets_singers)

In [86]:
gen = data_gen()
gen_2 = data_gen_new()

In [90]:
feats_targs, f0, phos, singers = next(gen)
feats_targs_2, f0_2, phos_2, singers_2 = next(gen_2)

6
12
18
24
30


In [91]:
print("features:", feats_targs.shape)
print("targets:", f0.shape)
print("phonemes", phos.shape)
print("singers", singers.shape)

features: (30, 128, 66)
targets: (30, 128, 1)
phonemes (30, 128)
singers (30,)


In [92]:
print("features:", feats_targs_2.shape)
print("targets:", f0_2.shape)
print("phonemes", phos_2.shape)
print("singers", singers_2.shape)

features: (30, 128, 66)
targets: (30, 128, 1)
phonemes (30, 128)
singers (30,)


In [93]:
import json
def json_dump(filename, data, delimiter):
    with open(filename, "w") as f:
        json.dump(data.tolist(), f)

In [94]:
# Saving processed batch data as CSV.

json_dump("feats_targs.json", feats_targs_2, delimiter=",")
json_dump("f0.json", f0_2, delimiter=",")
json_dump("phonemes.json", phos_2, delimiter=",")
json_dump("singers.json", singers_2, delimiter=",")

In [98]:
lmao = json.load(open("feats_targs.json",))
lmao2 = np.asarray(lmao)

(30, 128, 66)

In [178]:
feats_targs

array([[[0.7907194 , 0.815788  , 0.72727805, ..., 0.8085033 ,
         0.43155187, 0.        ],
        [0.7911183 , 0.81184375, 0.72891164, ..., 0.8244696 ,
         0.42919207, 0.        ],
        [0.7880935 , 0.81342137, 0.7260719 , ..., 0.8458469 ,
         0.43112615, 0.        ],
        ...,
        [0.8009909 , 0.81090057, 0.71474034, ..., 0.83798164,
         0.33084872, 0.        ],
        [0.79967964, 0.80171436, 0.71683174, ..., 0.732871  ,
         0.3295193 , 0.        ],
        [0.7951636 , 0.797104  , 0.71609837, ..., 0.8715859 ,
         0.3249046 , 0.        ]],

       [[0.7770126 , 0.7874902 , 0.72677433, ..., 0.8303036 ,
         0.41547403, 0.        ],
        [0.7758331 , 0.7969263 , 0.72191685, ..., 0.775331  ,
         0.4162075 , 0.        ],
        [0.7787678 , 0.8038338 , 0.72580034, ..., 0.81527835,
         0.41995266, 0.        ],
        ...,
        [0.78156394, 0.77894586, 0.71285444, ..., 0.7511721 ,
         0.33657902, 0.        ],
        [0.7

In [179]:
targets_f0_1

array([[[0.43155187],
        [0.42919207],
        [0.43112615],
        ...,
        [0.33084872],
        [0.3295193 ],
        [0.3249046 ]],

       [[0.41547403],
        [0.4162075 ],
        [0.41995266],
        ...,
        [0.33657902],
        [0.34096664],
        [0.34171903]],

       [[0.38928926],
        [0.38880435],
        [0.38821414],
        ...,
        [0.41054234],
        [0.4139085 ],
        [0.41354603]],

       ...,

       [[0.03449736],
        [0.16159132],
        [0.28868514],
        ...,
        [0.22230506],
        [0.22295503],
        [0.22360489]],

       [[0.2189226 ],
        [0.23632209],
        [0.25372148],
        ...,
        [0.29373568],
        [0.29663765],
        [0.29893878]],

       [[0.38884866],
        [0.38957217],
        [0.389174  ],
        ...,
        [0.23023984],
        [0.22713709],
        [0.22268467]]], dtype=float32)

In [180]:
phos

array([[21, 21, 21, ..., 15, 15, 15],
       [21, 21, 21, ..., 15, 15, 15],
       [15, 15, 15, ..., 14, 14, 14],
       ...,
       [ 0,  0, 26, ..., 19, 19, 19],
       [ 0,  0,  0, ..., 33, 33, 33],
       [32, 32, 32, ...,  1,  1,  1]])

In [176]:
singers

array([3, 3, 3, 3, 3, 3, 8, 8, 8, 8, 8, 8, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
       6, 6, 2, 2, 2, 2, 2, 2])

In [80]:
def policeman(mode = 'Train', sec_mode = 0):

    ## ADJUST WHATEVER IS IN VOC_LIST TO CHANGE NUMBER OF SAMPLES
    voc_list = [x for x in os.listdir(voice_dir) if x.endswith('.hdf5')]
    #print(voc_list)

    stat_file = h5py.File(config.stat_dir+'stats.hdf5', mode='r')
    max_feat = np.array(stat_file["feats_maximus"])
    min_feat = np.array(stat_file["feats_minimus"])
    stat_file.close()

    max_files_to_process = int(config.batch_size/config.samples_per_file)
    # max_files_to_process = 1

    if mode == "Train":
        num_batches = config.batches_per_epoch_train
        if sec_mode == 0:
            file_list = voc_list

    else: 
        # num_batches = config.batches_per_epoch_val
        # file_list = val_list
        return

    for k in range(num_batches):
        if sec_mode == 1:
            if np.random.rand(1)<config.aug_prob:
                file_list = voc_list
            else:
                file_list = voc_list
        

        feats_targs = []
        targets_f0_1 = []
        targets_singers = []
        pho_targs = []

        criminals = set()

        # start_time = time.time()
        if k == num_batches-1 and mode =="Train":
            file_list = voc_list

        for voc_index in range(len(voc_list)):
            #randomly choose some file to process
            voc_to_open = file_list[voc_index]
            voc_file = h5py.File(voice_dir + voc_to_open, "r")
            feats = np.array(voc_file['feats'])

            f0 = feats[:,-2]

            med = np.median(f0[f0 > 0])

            f0[f0==0] = med
            f0_nor = (f0 - min_feat[-2])/(max_feat[-2]-min_feat[-2])
            feats = (feats-min_feat)/(max_feat-min_feat)
            feats[:,-2] = f0_nor
            if feats.max()>1.0 and feats.min()<0.0:
                singer_name = voc_to_open.split('_')[1]
                criminals.add(singer_name)
        
    return criminals
                

        #     if voc_to_open.startswith('nus'):
        #         if not  "phonemes" in voc_file:
        #             print(voc_file)
        #             Flag = False
        #         else: 
        #             Flag = True
        #             pho_target = np.array(voc_file["phonemes"])
        #             singer_name = voc_to_open.split('_')[1]
        #             singer_index = config.singers.index(singer_name)
        #             print("singer", singer_name, singer_index)
        #     else:
        #         Flag = False

        #     # there are 6 samples per file
        #     for j in range(config.samples_per_file):
        #             # randomly get a WINDOW of 128 frames for each file.
        #             # each 128 frames is a sample
        #             voc_idx = np.random.randint(0,len(feats)-config.max_phr_len)
        #             print("start", voc_idx, "end", voc_idx+config.max_phr_len)
        #             targets_f0_1.append(f0_nor[voc_idx:voc_idx+config.max_phr_len])
        #             if Flag:
        #                 pho_targs.append(pho_target[voc_idx:voc_idx+config.max_phr_len])
        #                 targets_singers.append(singer_index)

        #             feats_targs.append(feats[voc_idx:voc_idx+config.max_phr_len])

        # targets_f0_1 = np.expand_dims(np.array(targets_f0_1), -1)

        # feats_targs = np.array(feats_targs)

        # assert feats_targs.max()<=1.0 and feats_targs.min()>=0.0, "offending singers: " + targets_singers 

        # yield feats_targs, targets_f0_1, np.array(pho_targs), np.array(targets_singers)

In [181]:
fucktards = policeman()
fucktards

KeyboardInterrupt: 